<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguaje Natural

## Taller #5: TF-IDF
`Fecha de entrega: Marzo 25, 2021. (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

In [12]:
import re
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
stopwords_sp = stopwords.words('spanish')

from nltk.stem.snowball import SnowballStemmer
spanishStemmer=SnowballStemmer("spanish")

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_distances

# Punto 1: Pre-Procesamiento

- `[9 pts]` Leer el archivo `bob_esponja.csv` usando `pandas`
- `[9 pts]` Crear una nueva columna con el texto en minúscula, sin caracteres especiales ni números, sin palabras vacias y hacer stemming de las palabras

In [110]:
# Leer archivo
df = pd.read_csv("bob_esponja.csv",encoding='utf-8')

def PreProcesado(column): 
    column = column.lower()
    column = re.sub(r"[\W\d_]+", " ", column)
    column = [palabra for palabra in column.split() if palabra not in stopwords_sp]
    column = " ".join(column)
    return column

In [112]:
#Nueva Columna
df['Texto_Pre_procesado'] = df['Descripción'].apply(lambda texto: pre_procesado(texto))
df

,Personaje,Descripción,Texto_Pre_procesado
0,Bob Esponja Pantalones Cuadrados,Bob Esponja es una esponja de mar con forma re...,bob esponja esponja mar forma rectangular colo...
1,Patricio Estrella,"Es una estrella de mar de color rosa, el mejor...",estrella mar color rosa mejor amigo bob esponj...
2,Calamardo Tentáculos,Calamardo Tentáculos es un personaje principal...,calamardo tentáculos personaje principal bob e...
3,Arenita Mejillas,Arenita Mejillas (Sandy Cheeks en inglés) en u...,arenita mejillas sandy cheeks inglés personaje...
4,Eugenio H. Cangrejo,Don Eugenio H. Cangrejo (normalmente llamado S...,don eugenio h cangrejo normalmente llamado señ...
5,Sheldon J. Plankton,"Sheldon J. Plankton, o simplemente Plankton, e...",sheldon j plankton simplemente plankton dos an...
6,Karen Plankton,Karen Plankton es uno de los dos principales a...,karen plankton dos principales antagonistas bo...
7,Perlita Cangrejo,"Perla ""Perlita"" Cangrejo es un personaje princ...",perla perlita cangrejo personaje principal bob...
8,Sra. Puff,La Señora Puff es un personaje principal de Bo...,señora puff personaje principal bob esponja ma...
9,Gary el Caracol,Gary el Caracol es un personaje principal de B...,gary caracol personaje principal bob esponja q...


# Punto 2: TF-IDF

- `[16 pts]` Crear la matriz TF-IDF

In [120]:
# tfidf_matrix
from sklearn.feature_extraction.text import TfidfVectorizer


vector = TfidfVectorizer()
tfidf = vector.fit_transform(df["Texto_Pre_procesado"].values)

tfidf_tabla = pd.DataFrame(tfidf.toarray())
tfidf_tabla.columns = vector.get_feature_names()
tfidf_tabla.index = df["Personaje"]
tfidf_tabla = tfidf_tabla.T
tfidf_tabla.round(5)

Personaje,Bob Esponja Pantalones Cuadrados,Patricio Estrella,Calamardo Tentáculos,Arenita Mejillas,Eugenio H. Cangrejo,Sheldon J. Plankton,Karen Plankton,Perlita Cangrejo,Sra. Puff,Gary el Caracol
abajo,0.05871,0.0,0.00000,0.00000,0.00000,0.0,0.00000,0.0,0.00000,0.00000
aborrece,0.00000,0.0,0.09071,0.00000,0.00000,0.0,0.00000,0.0,0.00000,0.00000
acaba,0.00000,0.0,0.09071,0.00000,0.00000,0.0,0.00000,0.0,0.00000,0.00000
accidente,0.04990,0.0,0.00000,0.00000,0.04553,0.0,0.00000,0.0,0.00000,0.00000
aceptó,0.00000,0.0,0.00000,0.00000,0.00000,0.0,0.06391,0.0,0.00000,0.00000
...,...,...,...,...,...,...,...,...,...,...
zapatos,0.04366,0.0,0.00000,0.00000,0.00000,0.0,0.00000,0.0,0.09168,0.06909
árbol,0.00000,0.0,0.00000,0.08068,0.00000,0.0,0.00000,0.0,0.00000,0.00000
éste,0.00000,0.0,0.00000,0.00000,0.00000,0.0,0.00000,0.0,0.00000,0.09289
única,0.05871,0.0,0.00000,0.00000,0.00000,0.0,0.00000,0.0,0.00000,0.00000


# Punto 3: Distancia del coseno
- `[12 pts]` Calcular la distancia del coseno entre cada una de los personajes
- `[2 pts]` ¿Cuáles son los personajes más parecidos?
- `[2 pts]` ¿Cuáles son los personajes más diferentes?

In [123]:
# dist_cos
from sklearn.metrics.pairwise import cosine_distances

Coseno_dis = cosine_distances(tfidf_tabla.T.values)
Coseno_dis = pd.DataFrame(Coseno_dis, 
                        columns = tfidf_tabla.columns, 
                        index = tfidf_tabla.columns)
print(Coseno_dis)

Personaje                         Bob Esponja Pantalones Cuadrados  \
Personaje                                                            
Bob Esponja Pantalones Cuadrados                          0.000000   
Patricio Estrella                                         0.850507   
Calamardo Tentáculos                                      0.927257   
Arenita Mejillas                                          0.871921   
Eugenio H. Cangrejo                                       0.881220   
Sheldon J. Plankton                                       0.901743   
Karen Plankton                                            0.957920   
Perlita Cangrejo                                          0.916038   
Sra. Puff                                                 0.882641   
Gary el Caracol                                           0.931071   

Personaje                         Patricio Estrella  Calamardo Tentáculos  \
Personaje                                                                   
Bob E

# ¿Cuáles son los personajes más parecidos?

In [133]:
print(Coseno_dis[Coseno_dis>0 ].min())
print('____________________________________________________')
print(Coseno_dis[Coseno_dis>0 ].idxmin())

Personaje
Bob Esponja Pantalones Cuadrados    0.850507
Patricio Estrella                   0.808538
Calamardo Tentáculos                0.808538
Arenita Mejillas                    0.871921
Eugenio H. Cangrejo                 0.704685
Sheldon J. Plankton                 0.704685
Karen Plankton                      0.808370
Perlita Cangrejo                    0.829929
Sra. Puff                           0.882641
Gary el Caracol                     0.914684
dtype: float64
____________________________________________________
Personaje
Bob Esponja Pantalones Cuadrados                   Patricio Estrella
Patricio Estrella                               Calamardo Tentáculos
Calamardo Tentáculos                               Patricio Estrella
Arenita Mejillas                    Bob Esponja Pantalones Cuadrados
Eugenio H. Cangrejo                              Sheldon J. Plankton
Sheldon J. Plankton                              Eugenio H. Cangrejo
Karen Plankton                                  

# ¿Cuáles son los personajes más diferentes?

In [134]:
print(Coseno_dis[Coseno_dis>0 ].max())
print('____________________________________________________')
print(Coseno_dis[Coseno_dis>0 ].idxmax())

Personaje
Bob Esponja Pantalones Cuadrados    0.957920
Patricio Estrella                   0.968058
Calamardo Tentáculos                0.972910
Arenita Mejillas                    0.983039
Eugenio H. Cangrejo                 0.938435
Sheldon J. Plankton                 0.990016
Karen Plankton                      0.991454
Perlita Cangrejo                    0.984115
Sra. Puff                           0.991454
Gary el Caracol                     0.978322
dtype: float64
____________________________________________________
Personaje
Bob Esponja Pantalones Cuadrados         Karen Plankton
Patricio Estrella                   Sheldon J. Plankton
Calamardo Tentáculos                   Perlita Cangrejo
Arenita Mejillas                              Sra. Puff
Eugenio H. Cangrejo                           Sra. Puff
Sheldon J. Plankton                           Sra. Puff
Karen Plankton                                Sra. Puff
Perlita Cangrejo                         Karen Plankton
Sra. Puff     